In [121]:
import os
os.chdir('../')
%pwd

'd:\\Python\\Industry level\\EndtoEnd combination'

In [122]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    test_data_path : Path
    label_encoder_path: Path
    vectorizer_path: Path
    ml_model_path: dict
    dl_model_path: Path

In [123]:
from resumeScreening.constants import *
from resumeScreening.utils.common import read_yaml,create_directories

In [124]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath =  PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation

        model_evaluation_config = ModelEvaluationConfig(
            test_data_path= Path(config.test_data_path),
            label_encoder_path= Path(config.label_encoder_path),
            vectorizer_path= Path(config.vectorizer_path),
            ml_model_path= {
                "logisticregression": Path(config.ml_model_path.logreg),
                "RandomForest": Path(config.ml_model_path.rf),
                "SVM": Path(config.ml_model_path.svc)
            },
            dl_model_path= Path(config.dl_model_path)
        )
        return model_evaluation_config

In [125]:
import pandas as pd
import pickle
from sklearn.metrics import accuracy_score,classification_report
import mlflow
import mlflow.sklearn
from resumeScreening import logger
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import to_categorical
import mlflow.keras

In [126]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def evaluate(self):
        #load test data
        test_df = pd.read_csv(self.config.test_data_path)
        X_test_text = test_df['Cleaned_Resume']
        y_test_labels = test_df['Category']

        #Load label encoder and vectorizer
        with open(self.config.label_encoder_path,'rb') as f:
            label_encoder = pickle.load(f)

        y_test_encoded = label_encoder.transform(y_test_labels)

        with open(self.config.vectorizer_path,'rb') as f:
            vectorizer = pickle.load(f)

        X_test_vectorized = vectorizer.transform(X_test_text)

        #Evaluate ML Models

        for name, path in self.config.ml_model_path.items():
            with open(path,'rb') as f:
                model = pickle.load(f)

            y_pred = model.predict(X_test_vectorized)
            acc = accuracy_score(y_test_encoded, y_pred)
            cr = classification_report(y_test_encoded,y_pred,target_names=label_encoder.classes_)
            logger.info(f"{name} Accuracy: {acc}")
            logger.info(f"Classifcation report: {cr}")

            with mlflow.start_run(run_name=f"{name}_eval"):
                mlflow.log_param("model_type", "ML")
                mlflow.log_metric("accuracy", acc)
                mlflow.sklearn.log_model(model, name)

    #Deep learning

        # Load vectorizer used during training
        with open(self.config.vectorizer_path, "rb") as f:
            vectorizer = pickle.load(f)

        X_test_vectorized = vectorizer.transform(X_test_text)
        X_test_dl = X_test_vectorized.toarray().astype("float32")

# Load trained DL model
        deep_model = load_model(self.config.dl_model_path)

# Confirm shapes match
        assert deep_model.input_shape[1] == X_test_dl.shape[1], \
        f"Shape mismatch: Model expects {deep_model.input_shape[1]}, but got {X_test_dl.shape[1]}"

# Prediction
        y_pred_probs = deep_model.predict(X_test_dl)
        y_pred_dl = y_pred_probs.argmax(axis=1)

# Evaluation
        acc_dl = accuracy_score(y_test_encoded, y_pred_dl)
        cr = classification_report(y_test_encoded, y_pred_dl, target_names=label_encoder.classes_)

        with mlflow.start_run(run_name="DL_eval"):
            mlflow.log_param("model_type", "DL")
            mlflow.log_metric("accuracy", acc_dl)
            mlflow.keras.log_model(deep_model, "DeepModel")
            

In [127]:
%pwd
os.chdir('d:\\Python\\Industry level\\EndtoEnd combination\\AI-Resume-Screening-with-ML-ops')

In [128]:
try:
    config = ConfigurationManager()
    model_eval_config = config.get_model_evaluation_config()
    evaluator = ModelEvaluation(config=model_eval_config)
    evaluator.evaluate()
except Exception as e:
    raise e


[2025-08-07 15:17:47,362: INFO: yaml file: config\config.yaml loaded successfully]
[2025-08-07 15:17:47,384: INFO: yaml file: params.yaml loaded successfully]
[2025-08-07 15:17:47,392: INFO: Created directory at: artifacts]
[2025-08-07 15:17:47,729: INFO: logisticregression Accuracy: 0.9844559585492227]
[2025-08-07 15:17:47,732: INFO: Classifcation report:                            precision    recall  f1-score   support

                 Advocate       1.00      0.33      0.50         3
                     Arts       1.00      1.00      1.00         6
       Automation Testing       1.00      1.00      1.00         5
               Blockchain       1.00      1.00      1.00         7
         Business Analyst       1.00      1.00      1.00         4
           Civil Engineer       1.00      1.00      1.00         9
             Data Science       1.00      1.00      1.00         5
                 Database       1.00      1.00      1.00         8
          DevOps Engineer       1.00 

2025/08/07 15:17:47 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/08/07 15:18:01 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[2025-08-07 15:18:01,943: INFO: RandomForest Accuracy: 0.9844559585492227]
[2025-08-07 15:18:01,943: INFO: Classifcation report:                            precision    recall  f1-score   support

                 Advocate       1.00      0.33      0.50         3
                     Arts       1.00      1.00      1.00         6
       Automation Testing       1.00      1.00      1.00         5
               Blockchain       1.00      1.00      1.00         7
         Business Analyst       1.00      1.00      1.00         4
           Civil Engineer       1.00      1.00      1.00         9
             Data Science       1.00      1.00      1.00         5
                 Database       1.00      1.00      1.00         8
          DevOps Engineer       1.00      0.93      0.96        14
         DotNet Developer       1.00      1.00      1.00         5
            ETL Developer       1.00      1.00      1.00         7
   Electrical Engineering       1.00      1.00      1.00         6

2025/08/07 15:18:02 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/08/07 15:18:08 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[2025-08-07 15:18:09,126: INFO: SVM Accuracy: 0.9844559585492227]
[2025-08-07 15:18:09,127: INFO: Classifcation report:                            precision    recall  f1-score   support

                 Advocate       1.00      0.33      0.50         3
                     Arts       1.00      1.00      1.00         6
       Automation Testing       1.00      1.00      1.00         5
               Blockchain       1.00      1.00      1.00         7
         Business Analyst       0.80      1.00      0.89         4
           Civil Engineer       1.00      1.00      1.00         9
             Data Science       1.00      1.00      1.00         5
                 Database       1.00      1.00      1.00         8
          DevOps Engineer       1.00      0.93      0.96        14
         DotNet Developer       1.00      1.00      1.00         5
            ETL Developer       1.00      1.00      1.00         7
   Electrical Engineering       1.00      1.00      1.00         6
        

2025/08/07 15:18:09 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/08/07 15:18:14 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[2025-08-07 15:18:15,332: WARNING: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step


d:\Python\Industry level\EndtoEnd combination\AI-Resume-Screening-with-ML-ops\resume\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
d:\Python\Industry level\EndtoEnd combination\AI-Resume-Screening-with-ML-ops\resume\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
d:\Python\Industry level\EndtoEnd combination\AI-Resume-Screening-with-ML-ops\resume\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no 